In [1]:
import os
import random
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler, Adam
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
from icecream import ic
from data_loader import *
from FFNN_model import *

In [2]:
class Labelled_Dataset(Dataset):
    def __init__(self, file_path):
        self.file = h5py.File(file_path, "r")
        self.measurements = self.file["measurements"]
        self.labels = self.file["labels"]

    def __len__(self):
        return self.measurements.shape[0]

    def __getitem__(self, idx):
        x = torch.tensor(self.measurements[idx])  # , dtype=torch.float32)
        y = torch.tensor(self.labels[idx])  # , dtype=torch.float32)
        return x, y

    def close(self):
        self.file.close()


file_path = "binary/reformatted_dataset.h5"

dataset = Labelled_Dataset(file_path)
print(len(dataset))  # PRINT
print(dataset[0][0].shape)
batch_size = 64

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(str(device))

N = 4  # number of images in whole picture
N_i = 25
N_h = 256
N_o = 1


device = torch.device("cpu")
# model
encoder = Encoder(N, N_i, N_h)
shared_encoder = SharedEncoder(N_i, N_h)
classifier = Classifier(N, N_h * 2, N_o)
model = MultiIonReadout(encoder, shared_encoder, classifier)

# Setting the hyperparameters
N_epochs = 50
lr = 1e-3
optimizer = Adam(model.parameters(), lr=lr)
schedule_params = {"factor": 1}
schedule = lr_scheduler.ConstantLR(optimizer, **schedule_params)
log_every = 1

# Training loop
for epoch in range(N_epochs):

    total_train_loss = 0
    for inputs, labels in train_loader:

        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        print("Shape of input tensor:", inputs.shape)  # PRINT
        print("Shape of labels tensor:", labels.shape)
        loss = model.bceloss(inputs, labels)

        loss.backward()
        optimizer.step()

    # Evaluation loop
    with torch.no_grad():
        total_loss = 0
        total_accuracy = 0
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            loss = model.bceloss(inputs, labels)
            accuracy = model.accuracy(inputs, labels)
            total_loss += loss.item()
            total_accuracy += accuracy.item()

        avg_loss = total_loss / len(val_loader)
        avg_accuracy = total_accuracy / len(val_loader)

    print(
        "\r Epoch {}/{}, Training Loss = {}, Val Loss = {}, Val Acc = {}".format(
            epoch + 1, N_epochs, loss.item(), avg_loss, avg_accuracy
        ),
        end="",
    )

2000
torch.Size([4, 25])
cpu
Shape of input tensor: torch.Size([64, 4, 25])
Shape of labels tensor: torch.Size([64, 4, 1])
Shape of input tensor in MultiIonReadout: torch.Size([64, 4, 25])
Shape of output tensor in MultiIonReadout: torch.Size([64, 4, 1])
Shape of input tensor 'y' in MultiIonReadout -> Encoder and Shared_Encoder: torch.Size([64, 100])
Shape of input 'X' tensor in IndexDependentDense: torch.Size([64, 100])


RuntimeError: einsum(): the number of subscripts in the equation (3) does not match the number of dimensions (2) for operand 1 and no ellipsis was given